# Stanford hyperspectral

We took an hyperspectral image of Stanford (and SF, Berkeley, up to Sacramento, down to Salinas) few weeks ago.

Download the data from [Telluric](https://telluric.satellogic.com/catalog/scene/newsat4_macro_cube_d455bfb4f126437bb85edb3a4754e6fd_0_4_4) and place it under `data/`

## Download the data

In [ ]:
#Authenticate on telluric
import requests

url = 'https://auth.telluric.satellogic.com/api-token-auth/'
payload = {'username':'stanfordhackathon','password':'hack2018'}

print("Getting token...")

r = requests.post(url, data=payload)
if r.status_code != 200:
    raise ValueError("Telluric response error: %s" %r.text) 

telluric_token="JWT "+r.json()['token']

print(telluric_token[0:10]+"*masked*")

In [ ]:
#get download id for the whole raster
url = 'https://telluric.satellogic.com/v2/scenes/download/'
scene_id = 'newsat4_macro_cube_d455bfb4f126437bb85edb3a4754e6fd_0_4_4'

header = {'authorization': telluric_token}
data = {'scene_id': scene_id,
        'async': 1}  # Important! This will prepare the download in the background for us

print("Requesting download...")

r = requests.get(url, params=data, headers=header)
if r.status_code != 200:
    raise ValueError("Telluric response error: %s" %r.text) 

response = r.json()
response

In [ ]:
#after a while, the download is ready
requests.get(r.json()['status_path'], headers=header).json()

In [ ]:
#download raster to a file (<10 minutes with a good connection)
url = response['download_url']
filename = response['filename']
header = { 'authorization' : telluric_token}

# http://docs.python-requests.org/en/master/user/quickstart/#raw-response-content
r = requests.get(url, headers=header, stream=True)
if r.status_code != 200:
    raise ValueError("Telluric response error: %s" %r.text) 

with open(filename, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)

In [ ]:
#unzip the contents
import os
from zipfile import ZipFile

data_path = "data/satellogic/macro"
os.makedirs(data_path)

with ZipFile(filename, 'r') as fp:
    fp.extractall(data_path)

---

In [ ]:
import os
import numpy as np

hypercube_folder="./data/satellogic/"
files=os.listdir(hypercube_folder) 

hfiles=np.sort([x for x in files if x[-6:]=='nm.tif'])

print("Number of Spectral bands: %i" % (len(hfiles)))
print(hfiles)


In [ ]:
#Read ALL bands, 
# mask them into the ROI and 
# save them into a single multiband cube
# MINIMIZE MEMORY FOOTPRINT

import fiona
import rasterio
from rasterio.mask import mask

#to mask the region we need both images in the same CRS
with fiona.open("./Stanford-SF.geojson", "r") as shapefile:
    geoms = [feature["geometry"] for feature in shapefile]

max_cast=40000
min_cast=0

with rasterio.open(hypercube_folder+hfiles[0]) as src:
    out_image, out_transform = mask(src, geoms, crop=True)
cube_dtype=np.uint8 #np.uint16 to have more spectral flux resolution.
cube=np.zeros((np.shape(out_image.data)[1], np.shape(out_image.data)[2],len(hfiles)),dtype=np.uint8)


print("Reading files...",end='')
for i in np.arange(len(hfiles[:])):
    file=hfiles[i]
    print(file,end=", "),
    with rasterio.open(hypercube_folder+file) as src:
        out_image, out_transform = mask(src, geoms, crop=True)
        if out_image.data.dtype == cube.dtype:
            cube[:,:,i]=out_image.data
        else:
            #cast to uint8, to save space
            cube[:,:,i]=((np.clip(out_image.data,min_cast,max_cast)-min_cast)/(max_cast-min_cast))*np.iinfo(cube.dtype).max
print("")

In [ ]:
#plot the data
%matplotlib notebook
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def macro_plot(c):
    wavelengths = [int(a[10:13]) for a in hfiles]

    i=int(c.shape[2]/2)    
    x=int(c.shape[0]/2)
    y=int(c.shape[1]/2)
    
    fig=plt.figure()
    im=plt.subplot(121)
    s=plt.subplot(122)

    im.cla()
    im.imshow(c[:,:,i])

    red=[620,750]
    green=[495,570]
    blue=[450,495]
    def spectra(i,x,y):
        s.cla()
        s.plot(wavelengths,c[x,y,:]/255,'o-')
        s.axvline(x=wavelengths[i],linestyle=':',color='black')
    
    #add RGB reference
        for p in [
        patches.Rectangle(
            (red[0], 0), red[1]-red[0], 1,
            alpha=.1,Color='red'
        ),
        patches.Rectangle(
            (green[0], 0), green[1]-green[0], 1,
            alpha=.1,Color='green'
        ),
        patches.Rectangle(
            (blue[0], 0), blue[1]-blue[0], 1,
            alpha=.1,Color='blue'
        ),
    ]:
            s.add_patch(p)
        plt.show()

    def onclick(event):
        y=int(event.xdata)
        x=int(event.ydata)
        i=i_slider.value
        spectra(i,x,y)
    im.figure.canvas.mpl_connect('button_press_event', onclick)
    

    def spectrogram(i):
        im.imshow(c[:,:,i], cmap='gray')
        spectra(i,x,y)

    spectrogram(i)
    i_slider = widgets.IntSlider(min=0,
                             max=c.shape[2]-1,
                             step=1,
                             value=c.shape[2]/2,
                             description='Channel')
    interact(spectrogram, i=i_slider)
macro_plot(cube)


In [ ]:
from spectral import *
k=30
loops=10
(m, c) = kmeans(cube[:,:,1:], k, loops)

In [ ]:
import matplotlib.cm as cm
km=plt.figure()

base = plt.cm.get_cmap(cm.jet)
cmap=base.from_list('', base(np.linspace(0, 1, k)), k)

plt.imshow(m,cmap=cmap)
plt.colorbar